In [2]:
# imports
import os
import random
import string
import base64
import json
import signal
from dotenv import load_dotenv
from requests import post, get
from requests_oauthlib import OAuth2Session

In [2]:
load_dotenv()
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
redirect_url = 'http://localhost:3000'

In [ ]:
def getToken():
    auth_str = client_id + ":" + client_secret
    auth_bytes = auth_str.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }

    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    
    # json_formatted_str = json.dumps(json_result, indent=2)
    # print(json_formatted_str)

    token = json_result["access_token"]
    return token

getToken()